<a href="https://colab.research.google.com/github/chanlenium/Python/blob/main/CIS%20eFAS%20Industry%20DB%20Scraping/NTS/%EA%B5%AD%EC%84%B8%EC%B2%AD_%ED%9C%B4%ED%8F%90%EC%97%85%EC%83%81%ED%83%9C%EC%A1%B0%ED%9A%8C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [47]:
# 사업자번호 판단 로직
import pandas as pd
import requests
import re

df = pd.read_csv("BRNO_2303.csv", dtype = object)  # csv파일을 String(object)으로 읽음

juminRulePattern = '\d{2}([0]\d|[1][0-2])([0][1-9]|[1-2]\d|[3][0-1])*[1-4]\d{6}'
df['isJuminNo'] = df.BRNO.str.match(pat=f'({juminRulePattern})')
df = df.loc[df["isJuminNo"] == False]  # 주민번호가 아니라고 판단되는 것들만 추출

filteredDf = pd.DataFrame()
filteredDf['b_no'] = df['BRNO'].str.slice(3, 13)
filteredDf

,b_no
5685,1076100173
9877,2096340173
12360,5078820013
20980,2015100178
31202,9888888888
...,...
5185226,9921183797
5185227,9922509061
5185228,9927888908
5185229,9929261827


In [ ]:
import pandas as pd
import requests
import re

headers = {
    'accept': 'application/json',
    'Authorization': '6No4JLBZPafGmvs0YQ/WiHSNqVrvqqAox0frrHQJbt/vc6QT4xzf2Idnc9CaktyAdlI3FYGILmdhVIH9u5ck2Q==',
    'Content-Type': 'application/json',
}

params = {
    'serviceKey': '6No4JLBZPafGmvs0YQ/WiHSNqVrvqqAox0frrHQJbt/vc6QT4xzf2Idnc9CaktyAdlI3FYGILmdhVIH9u5ck2Q==',
}

df = pd.read_csv("BRNO_2303.csv", dtype = object)  # csv파일을 String(object)으로 읽음
juminRulePattern = '\d{2}([0]\d|[1][0-2])([0][1-9]|[1-2]\d|[3][0-1])[1-4]\d{6}'  # 주민번호 판단을 위한 정규식
# \d{2} : 맨앞 정수 2자리(생년)은 어떤 정수값이 와도 상관없음
# ([0]\d|[1][0-2]) : 첫자리가 0인 경우는 뒤에 어떤 정수가 와도 괜찮음. 첫자리가 1인 경우 뒷자리는 0, 1, 2만 올 수 있음 (01-12 생월을 표현)
# ([0][1-9]|[1-2]\d|[3][0-1]) : 생일은 첫자리가 0이면 뒷자리가 0이 될 경우 0일이 되기 때문에 0 다음에는 1-9만 올 수 있음
# [1-4] : 주민번호 뒷자리 첫번째 숫자는 1~4만 갖는다
# \d{6} : 주민번호 첫자리를 제외한 숫자는 총 6자리다
df['isJuminNo'] = df.BRNO.str.match(pat=f'({juminRulePattern})')  # 주민번호 판단을 위한 정규식 적용
df = df.loc[df["isJuminNo"] == False]  # 주민번호가 아니라고 판단되는 것들만 추출

filteredDf = pd.DataFrame()
filteredDf['b_no'] = df['BRNO'].str.slice(3, 13)  # 앞의 3자리 제거(일례로 '999'제거)

dfReturnValue = pd.DataFrame()

print(f'전체 검사 대상 사업자번호 수는 {len(filteredDf)} 입니다')
start = 1500000
end = min(3000000, len(filteredDf) + 1)
interval = 100

for n in range(start, end, interval): # 100개씩 읽음 (start, end, interval)
    print(n)
    dfBRNO = filteredDf.iloc[n:n+100, :]
    json_data = dfBRNO.to_dict('list')
    response = requests.post('https://api.odcloud.kr/api/nts-businessman/v1/status', params=params, headers=headers, json=json_data)
    # print(response.status_code)
    jsResponse = response.json()['data']
    dfReturnValue =  pd.concat([dfReturnValue, pd.DataFrame(jsResponse)], axis=0)

dfReturnValue.reset_index(inplace=True)
dfReturnValue.to_excel(f"결과_{start}_{end}.xlsx")

In [ ]:
dfReturnValue.loc[dfReturnValue['b_stt_cd']=='01']

In [ ]:
dfReturnValue.loc[dfReturnValue['b_stt_cd']=='02']

In [ ]:
dfReturnValue.loc[dfReturnValue['b_stt_cd']=='03']